# 必要ライブラリの読み込み

In [11]:
#ライブラリ読み込み
import lightgbm as lgb
import numpy as np
import sklearn.datasets
import sklearn.metrics
from sklearn.model_selection import train_test_split

import optuna

pd.set_option('display.max_columns',200)
pd.set_option('display.max_rows',200)

# 必要データの読み込み

In [9]:
#データ読み込み

#データ定義
train_input= pd.read_csv('train_0806best.csv')
test_input= pd.read_csv('test_0806best.csv')

In [16]:
category_columns = ['city1',
 'city2',
 'city3',
 '階数',
 '部屋数',
 'jukyo',
 'yoto1',
 'yoto2',
 'josui',
 'gesui',
 'gas',
 'usui',
 'road1_hk',
 'road1_sb',
 'road2_hk',
 'road2_sb',
 'road3_hk',
 'road3_sb',
 'road4_hk',
 'road4_sb',
 'hw_status',
 'toshikuiki1',
 'toshikuiki2',
 'kodochiku',
 'bokachiiki',
 'hokakisei1',
 'hokakisei2',
 'hokakisei3',
 'hokakisei4',
 'kinshijiko',
 'rosen_nm1',
 'eki_nm1',
 'bas_toho1',
 'bastei_nm1',
 'rosen_nm2',
 'eki_nm2',
 'bas_toho2',
 'bastei_nm2',
 'levelplan',
 'road_st',
 'setsudo_hi',
 'setsudo_kj',
 'jigata',
 'hiatari',
 'kobetsu1',
 'kobetsu2',
 'kobetsu3',
 'kobetsu4']

In [18]:
#LightGBMでのカテゴリ扱いをする変数を指定
for column in category_columns:
    train_input[column] = train_input[column].astype('category')
    test_input[column] = test_input[column].astype('category')

In [32]:
y_true = np.array([3, -0.5, 2, 7])
y_pred = np.array( [2.5, 0.0, 2, 8])

In [34]:
np.mean((y_pred  - y_true )/y_true )

-0.255952380952381

In [38]:
y_pred.shape

(4,)

In [28]:
(y_pred - y_true)/y_true 

TypeError: unsupported operand type(s) for -: 'list' and 'list'

In [51]:
import lightgbm as lgb
import numpy as np
import sklearn.datasets
import sklearn.metrics
from sklearn.model_selection import train_test_split

import optuna


def objective(trial):
    data = train_input.drop(['id','pj_no','keiyaku_pr'],axis = 1)
    target = train_input[['keiyaku_pr']]
    train_x, test_x, train_y, test_y = train_test_split(data, target, test_size=0.2)
    dtrain = lgb.Dataset(train_x, label=train_y)

    param = {
            'task' : 'train',
            'boosting_type' : 'dart',
            'objective' : 'regression',
            'metric' : {'mape'},
            'num_leaves' : trial.suggest_int('num_leaves', 10, 100),
            'learning_rate' : trial.suggest_loguniform('learning_rate', 1e-8, 1.0),
            'n_estimators' : 1000,
            'feature_fraction' : 0.5,
            'bagging_fraction' : 0.8,
            'bagging_freq': 5,
            'verbose' : 0,
            'drop_rate' : trial.suggest_loguniform('drop_rate', 1e-8, 1.0),
            'skip_drop' : trial.suggest_loguniform('skip_drop', 1e-8, 1.0)
    }

    gbm = lgb.train(param, dtrain)
    preds = gbm.predict(test_x)
    print(preds.shape)
    print(test_y['keiyaku_pr'].values)
#     pred_labels = np.rint(preds)
    test_y = test_y['keiyaku_pr'].values
    
#     mae = sklearn.metrics.mean_absolute_error(test_y,preds)
#     return mae
    
    mape = np.mean(abs(preds  - test_y)/test_y ) * 100
    
    print('hello')
    return mape


if __name__ == '__main__':
    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=100)

    print('Number of finished trials: {}'.format(len(study.trials)))

    print('Best trial:')
    trial = study.best_trial

    print('  Value: {}'.format(trial.value))

    print('  Params: ')
    for key, value in trial.params.items():
        print('    {}: {}'.format(key, value))

/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:118: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


(1292,)
[27800000 30000000 29800000 ... 30800000 27800000 32000000]
hello


[I 2019-08-06 15:06:22,887] Finished trial#0 resulted in value: 7.016202808403676. Current best value is 7.016202808403676 with parameters: {'num_leaves': 46, 'learning_rate': 0.06510192767154657, 'drop_rate': 0.00014132265634717668, 'skip_drop': 0.2843873897773534}.


(1292,)
[43000000 21496297 16800000 ... 17900000 29300000 31800000]
hello


[I 2019-08-06 15:06:35,505] Finished trial#1 resulted in value: 9.38825949836608. Current best value is 7.016202808403676 with parameters: {'num_leaves': 46, 'learning_rate': 0.06510192767154657, 'drop_rate': 0.00014132265634717668, 'skip_drop': 0.2843873897773534}.


...

(1292,)
[42000000 45980000 25000000 ... 23300000 31000000 29500000]
hello


[I 2019-08-06 15:25:24,822] Finished trial#98 resulted in value: 7.148883521178065. Current best value is 6.543919431673996 with parameters: {'num_leaves': 89, 'learning_rate': 0.020995848649768882, 'drop_rate': 2.0304129928302412e-07, 'skip_drop': 0.003965941780322623}.


(1292,)
[22000000 15000000 27400000 ... 27800000 10800000 38000000]
hello


[I 2019-08-06 15:25:33,642] Finished trial#99 resulted in value: 13.206510086409695. Current best value is 6.543919431673996 with parameters: {'num_leaves': 89, 'learning_rate': 0.020995848649768882, 'drop_rate': 2.0304129928302412e-07, 'skip_drop': 0.003965941780322623}.


Number of finished trials: 100
Best trial:
  Value: 6.543919431673996
  Params: 
    num_leaves: 89
    learning_rate: 0.020995848649768882
    drop_rate: 2.0304129928302412e-07
    skip_drop: 0.003965941780322623


# データ整形

In [ ]:
#必要な列だけ残す
X = train_input.drop(['id','pj_no','keiyaku_pr'],axis = 1)
X_test = test_input.drop(['id','pj_no'],axis = 1)
y = train_input[['keiyaku_pr']]

Parameter_df =pd.DataFrame()
MAPE_df = pd.DataFrame()
MAPE_train_df =pd.DataFrame()
Submit_df = test_input[['id']].copy(deep = True)
Submit_df['MAPE_sum'] = 0

#グループ分けの定義
groups = train_input['pj_no'].array


#gkf = GroupShuffleSplit(test_size =0.2,random_state=0)
kf = KFold(n_splits= 5,shuffle=True,random_state=0)

i = 0
#for train,valid  in tqdm(gkf.split(X, y,groups=groups)):
for train,valid in tqdm(kf.split(X,y)):
    X_train = X.iloc[train.tolist()]
    X_valid = X.iloc[valid.tolist()]
    y_train = y.iloc[train.tolist()]
    y_valid = y.iloc[valid.tolist()]

#     # 学習データと検証データに分割する
#     #X_train, X_valid, y_train, y_valid = train_test_split(X, y,test_size = 0.2,shuffle = True,random_state = i+10)
#     X_train, X_valid, y_train, y_valid = GroupShuffleSplit(X, y,groups=train_input['pj_no'].array,test_size = 0.2,shuffle = True,random_state = i+10)

    # 上記のパラメータでモデルを学習する

    #'''
#     model = LGBMRegressorCV(
#                             n_estimators = 100,
#                             learning_rate = 0.1,
#                             n_trials = 100,
#                             n_iter_no_change=10,
#                             random_state=0,
#                             timeout = 60)
#     #'''

#     #ハイパーパラメータチューニングしない場合はこっち
#     #model = lgb.LGBMRegressor(random_state=10)

#     #学習
#     model.fit(X_train, y_train,categorical_feature=important_categorical_columns)

    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_eval = lgb.Dataset(X_valid, y_valid, reference=lgb_train)
    # LightGBM parameters
    params = {
            'task' : 'train',
            'boosting_type' : 'dart',
            'objective' : 'regression',
            'metric' : {'mape'},
            'num_leaves' :89,
            'learning_rate' : 0.020995848649768882,
            'n_estimators' : 3000,
            'feature_fraction' : 0.5,
            'bagging_fraction' : 0.8,
            'bagging_freq': 5,
            'verbose' : 0,
            'drop_rate' : 2.0304129928302412e-07,
            'skip_drop' : 0.003965941780322623
    }


    # train
    model = lgb.train(params,
                lgb_train,
                num_boost_round=4000,
                valid_sets=lgb_eval,
                early_stopping_rounds=300)

    #選ばれたハイパーパラメータの記録
    #temp_parameter_df = pd.DataFrame(model.best_params_.items()).set_index(0).T
    #Parameter_df = Parameter_df.append(temp_parameter_df )

    # 検証データを予測する
    #y_pred = model.predict(X_valid)
    y_pred = model.predict(X_valid, num_iteration=model.best_iteration)
    y_pred_train = model.predict(X_train, num_iteration=model.best_iteration)

    #検証スコア算出
    y_pred_df = pd.DataFrame(y_pred)
    y_valid = y_valid.reset_index().drop('index',axis=1)
    compared = pd.concat([y_pred_df,y_valid ],axis = 1)
    compared['APE'] = abs((compared[0]-compared['keiyaku_pr'])/compared['keiyaku_pr'])
    
    #検証スコア算出(train)
    y_pred_train_df = pd.DataFrame(y_pred_train)
    y_train = y_train.reset_index().drop('index',axis=1)
    compared_train = pd.concat([y_pred_train_df,y_train ],axis = 1)
    compared_train['APE'] = abs((compared_train[0]-compared_train['keiyaku_pr'])/compared_train['keiyaku_pr'])

    #検証スコアを記録
    temp_dict = {'MAPE':compared['APE'].mean()*100}
    temp_df = pd.DataFrame(temp_dict.items()).set_index(0).T
    MAPE_df = MAPE_df.append(temp_df)

    #検証スコアを記録
    temp_dict_train = {'MAPE':compared_train['APE'].mean()*100}
    temp_train_df = pd.DataFrame(temp_dict_train.items()).set_index(0).T
    MAPE_train_df = MAPE_train_df.append(temp_train_df)
    
    #予測を実行
    #y_test_pred = model.predict(X_test)
    y_test_pred = model.predict(X_test, num_iteration=model.best_iteration)
    y_test_pred_df = pd.DataFrame(y_test_pred)

    #提出用のデータフレーム作成
    temp_submit = pd.concat([test_input[['id']],y_test_pred_df],axis=1)
    temp_submit.rename(columns={0:'MAPE_'+str(i)},inplace = True)
    Submit_df = pd.merge(Submit_df,temp_submit,on='id')

    Submit_df['MAPE_sum'] = Submit_df['MAPE_sum'] + Submit_df['MAPE_'+str(i)]
    i = i+1
    
Submit_df['MAPE_mean'] = Submit_df['MAPE_sum']/(i)
    


0it [00:00, ?it/s]

[1]	valid_0's mape: 0.213302
[2]	valid_0's mape: 0.21028
[3]	valid_0's mape: 0.207286
[4]	valid_0's mape: 0.204356
[5]	valid_0's mape: 0.201579
[6]	valid_0's mape: 0.19862
[7]	valid_0's mape: 0.195822
[8]	valid_0's mape: 0.193263
[9]	valid_0's mape: 0.190543
[10]	valid_0's mape: 0.187869
[11]	valid_0's mape: 0.185436
[12]	valid_0's mape: 0.182867
[13]	valid_0's mape: 0.180515
[14]	valid_0's mape: 0.178127
[15]	valid_0's mape: 0.175755
[16]	valid_0's mape: 0.173496
[17]	valid_0's mape: 0.171145
[18]	valid_0's mape: 0.168969
[19]	valid_0's mape: 0.166986
[20]	valid_0's mape: 0.164876
[21]	valid_0's mape: 0.162788
[22]	valid_0's mape: 0.160836
[23]	valid_0's mape: 0.158874
[24]	valid_0's mape: 0.15704
[25]	valid_0's mape: 0.15525
[26]	valid_0's mape: 0.153445
[27]	valid_0's mape: 0.151736
[28]	valid_0's mape: 0.150045
[29]	valid_0's mape: 0.148484
[30]	valid_0's mape: 0.146867
[31]	valid_0's mape: 0.145306
[32]	valid_0's mape: 0.143731
[33]	valid_0's mape: 0.142354
[34]	valid_0's mape: 0.

[263]	valid_0's mape: 0.0766506
[264]	valid_0's mape: 0.0765875
[265]	valid_0's mape: 0.0765503
[266]	valid_0's mape: 0.0765132
[267]	valid_0's mape: 0.0765075
[268]	valid_0's mape: 0.076477
[269]	valid_0's mape: 0.0764737
[270]	valid_0's mape: 0.0764623
[271]	valid_0's mape: 0.0764231
[272]	valid_0's mape: 0.0763745
[273]	valid_0's mape: 0.0763646
[274]	valid_0's mape: 0.0763295
[275]	valid_0's mape: 0.0762976
[276]	valid_0's mape: 0.0762729
[277]	valid_0's mape: 0.07623
[278]	valid_0's mape: 0.0761857
[279]	valid_0's mape: 0.0761512
[280]	valid_0's mape: 0.0761183
[281]	valid_0's mape: 0.0760932
[282]	valid_0's mape: 0.076063
[283]	valid_0's mape: 0.0760486
[284]	valid_0's mape: 0.0760321
[285]	valid_0's mape: 0.0759992
[286]	valid_0's mape: 0.0759611
[287]	valid_0's mape: 0.0759292
[288]	valid_0's mape: 0.0759104
[289]	valid_0's mape: 0.0759006
[290]	valid_0's mape: 0.0758727
[291]	valid_0's mape: 0.0758711
[292]	valid_0's mape: 0.0758522
[293]	valid_0's mape: 0.0758074
[294]	valid_

[519]	valid_0's mape: 0.0738343
[520]	valid_0's mape: 0.0738298
[521]	valid_0's mape: 0.0738218
[522]	valid_0's mape: 0.0738068
[523]	valid_0's mape: 0.0738027
[524]	valid_0's mape: 0.0738044
[525]	valid_0's mape: 0.0737912
[526]	valid_0's mape: 0.0737821
[527]	valid_0's mape: 0.0737713
[528]	valid_0's mape: 0.0737584
[529]	valid_0's mape: 0.073754
[530]	valid_0's mape: 0.0737544
[531]	valid_0's mape: 0.0737539
[532]	valid_0's mape: 0.073749
[533]	valid_0's mape: 0.0737476
[534]	valid_0's mape: 0.0737388
[535]	valid_0's mape: 0.0737339
[536]	valid_0's mape: 0.0737448
[537]	valid_0's mape: 0.0737534
[538]	valid_0's mape: 0.0737619
[539]	valid_0's mape: 0.0737665
[540]	valid_0's mape: 0.0737587
[541]	valid_0's mape: 0.0737601
[542]	valid_0's mape: 0.0737627
[543]	valid_0's mape: 0.0737653
[544]	valid_0's mape: 0.0737676
[545]	valid_0's mape: 0.0737629
[546]	valid_0's mape: 0.0737579
[547]	valid_0's mape: 0.0737481
[548]	valid_0's mape: 0.0737508
[549]	valid_0's mape: 0.0737407
[550]	vali

[775]	valid_0's mape: 0.0735545
[776]	valid_0's mape: 0.0735538
[777]	valid_0's mape: 0.0735538
[778]	valid_0's mape: 0.0735591
[779]	valid_0's mape: 0.0735505
[780]	valid_0's mape: 0.0735411
[781]	valid_0's mape: 0.0735439
[782]	valid_0's mape: 0.0735425
[783]	valid_0's mape: 0.0735414
[784]	valid_0's mape: 0.0735506
[785]	valid_0's mape: 0.0735549
[786]	valid_0's mape: 0.0735545
[787]	valid_0's mape: 0.0735517
[788]	valid_0's mape: 0.0735552
[789]	valid_0's mape: 0.073561
[790]	valid_0's mape: 0.0735574
[791]	valid_0's mape: 0.0735625
[792]	valid_0's mape: 0.0735645
[793]	valid_0's mape: 0.0735617
[794]	valid_0's mape: 0.0735646
[795]	valid_0's mape: 0.0735651
[796]	valid_0's mape: 0.0735629
[797]	valid_0's mape: 0.0735584
[798]	valid_0's mape: 0.0735549
[799]	valid_0's mape: 0.0735516
[800]	valid_0's mape: 0.0735537
[801]	valid_0's mape: 0.0735497
[802]	valid_0's mape: 0.0735418
[803]	valid_0's mape: 0.0736107
[804]	valid_0's mape: 0.0736013
[805]	valid_0's mape: 0.0736051
[806]	val

...

[263]	valid_0's mape: 0.0715215
[264]	valid_0's mape: 0.0714958
[265]	valid_0's mape: 0.0714642
[266]	valid_0's mape: 0.0714465
[267]	valid_0's mape: 0.0714147
[268]	valid_0's mape: 0.0714041
[269]	valid_0's mape: 0.0713853
[270]	valid_0's mape: 0.0713477
[271]	valid_0's mape: 0.0713384
[272]	valid_0's mape: 0.0713204
[273]	valid_0's mape: 0.0713056
[274]	valid_0's mape: 0.0712662
[275]	valid_0's mape: 0.0712323
[276]	valid_0's mape: 0.0712008
[277]	valid_0's mape: 0.0711815
[278]	valid_0's mape: 0.0711455
[279]	valid_0's mape: 0.0711076
[280]	valid_0's mape: 0.0710845
[281]	valid_0's mape: 0.071056
[282]	valid_0's mape: 0.0710337
[283]	valid_0's mape: 0.07101
[284]	valid_0's mape: 0.0710004
[285]	valid_0's mape: 0.0709846
[286]	valid_0's mape: 0.0709659
[287]	valid_0's mape: 0.0709609
[288]	valid_0's mape: 0.0709286
[289]	valid_0's mape: 0.0708921
[290]	valid_0's mape: 0.0708809
[291]	valid_0's mape: 0.0708528
[292]	valid_0's mape: 0.0708517
[293]	valid_0's mape: 0.0708592
[294]	valid

[519]	valid_0's mape: 0.0685838
[520]	valid_0's mape: 0.068583
[521]	valid_0's mape: 0.0685853
[522]	valid_0's mape: 0.0685894
[523]	valid_0's mape: 0.068597
[524]	valid_0's mape: 0.0685863
[525]	valid_0's mape: 0.0685859
[526]	valid_0's mape: 0.0685951
[527]	valid_0's mape: 0.0685966
[528]	valid_0's mape: 0.0685945
[529]	valid_0's mape: 0.0685825
[530]	valid_0's mape: 0.0685862
[531]	valid_0's mape: 0.0685691
[532]	valid_0's mape: 0.0685608
[533]	valid_0's mape: 0.0685586
[534]	valid_0's mape: 0.0685536
[535]	valid_0's mape: 0.068553
[536]	valid_0's mape: 0.0685481
[537]	valid_0's mape: 0.0685361
[538]	valid_0's mape: 0.0685249
[539]	valid_0's mape: 0.0685104
[540]	valid_0's mape: 0.0684976
[541]	valid_0's mape: 0.0684975
[542]	valid_0's mape: 0.0684893
[543]	valid_0's mape: 0.0684777
[544]	valid_0's mape: 0.0684617
[545]	valid_0's mape: 0.0684546
[546]	valid_0's mape: 0.0684532
[547]	valid_0's mape: 0.0684387
[548]	valid_0's mape: 0.0684365
[549]	valid_0's mape: 0.0684437
[550]	valid

[776]	valid_0's mape: 0.0679829
[777]	valid_0's mape: 0.0679732
[778]	valid_0's mape: 0.0679724
[779]	valid_0's mape: 0.067967
[780]	valid_0's mape: 0.0679585
[781]	valid_0's mape: 0.0679527
[782]	valid_0's mape: 0.0679446
[783]	valid_0's mape: 0.0679464
[784]	valid_0's mape: 0.0679477
[785]	valid_0's mape: 0.0679454
[786]	valid_0's mape: 0.067943
[787]	valid_0's mape: 0.0679432
[788]	valid_0's mape: 0.0679363
[789]	valid_0's mape: 0.0679594
[790]	valid_0's mape: 0.0679508
[791]	valid_0's mape: 0.0679495
[792]	valid_0's mape: 0.0679524
[793]	valid_0's mape: 0.0679414
[794]	valid_0's mape: 0.0679358
[795]	valid_0's mape: 0.0679358
[796]	valid_0's mape: 0.0679389
[797]	valid_0's mape: 0.0679389
[798]	valid_0's mape: 0.067937
[799]	valid_0's mape: 0.0679387
[800]	valid_0's mape: 0.0679355
[801]	valid_0's mape: 0.0679382
[802]	valid_0's mape: 0.0679384
[803]	valid_0's mape: 0.0679565
[804]	valid_0's mape: 0.0679529
[805]	valid_0's mape: 0.0679529
[806]	valid_0's mape: 0.0679512
[807]	valid

[1031]	valid_0's mape: 0.0679147
[1032]	valid_0's mape: 0.0679111
[1033]	valid_0's mape: 0.0679119
[1034]	valid_0's mape: 0.0679163
[1035]	valid_0's mape: 0.0679129
[1036]	valid_0's mape: 0.0679113
[1037]	valid_0's mape: 0.0679104
[1038]	valid_0's mape: 0.0679088
[1039]	valid_0's mape: 0.0679069
[1040]	valid_0's mape: 0.0679112
[1041]	valid_0's mape: 0.0679099
[1042]	valid_0's mape: 0.0679107
[1043]	valid_0's mape: 0.0679165
[1044]	valid_0's mape: 0.067916
[1045]	valid_0's mape: 0.0679209
[1046]	valid_0's mape: 0.067921
[1047]	valid_0's mape: 0.0679153
[1048]	valid_0's mape: 0.067916
[1049]	valid_0's mape: 0.067917
[1050]	valid_0's mape: 0.0679117
[1051]	valid_0's mape: 0.0679162
[1052]	valid_0's mape: 0.0679118
[1053]	valid_0's mape: 0.0679104
[1054]	valid_0's mape: 0.06792
[1055]	valid_0's mape: 0.067925
[1056]	valid_0's mape: 0.0679216
[1057]	valid_0's mape: 0.0679229
[1058]	valid_0's mape: 0.067918
[1059]	valid_0's mape: 0.0679135
[1060]	valid_0's mape: 0.0679095
[1061]	valid_0's m

[1280]	valid_0's mape: 0.0680976
[1281]	valid_0's mape: 0.0680935
[1282]	valid_0's mape: 0.0680902
[1283]	valid_0's mape: 0.0680919
[1284]	valid_0's mape: 0.0680903
[1285]	valid_0's mape: 0.0680905
[1286]	valid_0's mape: 0.0680893
[1287]	valid_0's mape: 0.0680841
[1288]	valid_0's mape: 0.0680873
[1289]	valid_0's mape: 0.0680892
[1290]	valid_0's mape: 0.0680904
[1291]	valid_0's mape: 0.0680885
[1292]	valid_0's mape: 0.0680845
[1293]	valid_0's mape: 0.0680882
[1294]	valid_0's mape: 0.0680868
[1295]	valid_0's mape: 0.0680883
[1296]	valid_0's mape: 0.0680907
[1297]	valid_0's mape: 0.0680911
[1298]	valid_0's mape: 0.0680896
[1299]	valid_0's mape: 0.0680955
[1300]	valid_0's mape: 0.0680976
[1301]	valid_0's mape: 0.0680986
[1302]	valid_0's mape: 0.0681024
[1303]	valid_0's mape: 0.0681027
[1304]	valid_0's mape: 0.0681016
[1305]	valid_0's mape: 0.0680981
[1306]	valid_0's mape: 0.0680994
[1307]	valid_0's mape: 0.0680985
[1308]	valid_0's mape: 0.068099
[1309]	valid_0's mape: 0.0681051
[1310]	vali

In [52]:
#ハイパーパラメータ確認
Parameter_df

""


In [53]:
#各回でのMAPE検証値
MAPE_df

,MAPE
1,7.191017
1,6.723687
1,6.622930
1,6.932635
1,6.804854


In [54]:
MAPE_train_df

,MAPE
1,2.662512
1,2.680875
1,2.704113
1,2.704204
1,2.674498


In [55]:
compared['APE'].describe()

count    1291.000000
mean        0.068049
std         0.063479
min         0.000089
25%         0.025739
50%         0.053609
75%         0.096561
max         1.135550
Name: APE, dtype: float64

In [56]:
compared_train.sort_values('APE',ascending=False)

,0,keiyaku_pr,APE
1469,4.691172e+06,3500000,0.340335
3722,1.351600e+07,10800000,0.251481
2885,2.930324e+07,24000000,0.220968
537,1.589137e+07,19900000,0.201439
1527,8.612864e+06,7300000,0.179844
3719,1.970623e+07,17000000,0.159190
3718,1.970623e+07,17000000,0.159190
3498,1.954090e+07,17000000,0.149465
1712,2.524942e+07,29500000,0.144087
4832,1.827815e+07,16000000,0.142385


In [57]:
compared.sort_values('APE',ascending=False)

,0,keiyaku_pr,APE
1109,1.174552e+07,5500000,1.135550
435,2.706696e+07,18500000,0.463079
926,1.087808e+07,7900000,0.376972
1097,2.705065e+07,19990000,0.353209
486,2.116742e+07,31600000,0.330145
963,7.886088e+06,6000000,0.314348
638,1.559145e+07,11990000,0.300371
26,2.633339e+07,20300000,0.297211
578,5.901443e+07,46300000,0.274610
1270,2.251504e+07,17800000,0.264890


In [58]:
MAPE_df.mean()

0
MAPE    6.855025
dtype: float64

In [70]:
#結果の確認
Submit_df 

,id,MAPE_sum,MAPE_0,MAPE_1,MAPE_2,MAPE_3,MAPE_4,MAPE_mean
0,test_0000,9.783084e+07,1.904981e+07,2.032493e+07,1.925134e+07,1.948853e+07,1.971624e+07,1.956617e+07
1,test_0001,1.319030e+08,2.635308e+07,2.614885e+07,2.675512e+07,2.604056e+07,2.660535e+07,2.638059e+07
2,test_0002,1.259886e+08,2.535881e+07,2.549342e+07,2.539405e+07,2.462888e+07,2.511344e+07,2.519772e+07
3,test_0003,1.379103e+08,2.719893e+07,2.821971e+07,2.747368e+07,2.771393e+07,2.730409e+07,2.758207e+07
4,test_0004,1.321892e+08,2.665605e+07,2.617271e+07,2.663690e+07,2.661396e+07,2.610956e+07,2.643784e+07
5,test_0005,1.237402e+08,2.477051e+07,2.480942e+07,2.470154e+07,2.447721e+07,2.498154e+07,2.474804e+07
6,test_0006,1.333607e+08,2.698783e+07,2.662614e+07,2.654970e+07,2.700578e+07,2.619130e+07,2.667215e+07
7,test_0007,1.259793e+08,2.517997e+07,2.538658e+07,2.513862e+07,2.505475e+07,2.521942e+07,2.519587e+07
8,test_0008,1.311835e+08,2.633738e+07,2.608423e+07,2.661418e+07,2.565175e+07,2.649597e+07,2.623670e+07
9,test_0009,1.466839e+08,2.930456e+07,2.940274e+07,2.939854e+07,2.963082e+07,2.894723e+07,2.933678e+07


In [61]:
importance_df = pd.DataFrame([model.feature_importance(importance_type='gain'),X.columns]).T
importance_df.rename(columns={0:'importance',1:'feature_name'},inplace =True)
importance_df.set_index('feature_name',inplace=True)

In [62]:
importance_df.sort_values('importance',ascending = False)

,importance
feature_name,
koji_tatemenseki,1.43817e+18
koji_totimenseki,8.07062e+17
rosenka_hb,7.67261e+17
kijun_tatemenseki,7.58759e+17
kijun_totimenseki,3.40724e+17
tt_mseki,3.16716e+17
toho_kyori1,2.14758e+17
latitude,1.95784e+17
eki_nm1,1.95104e+17


In [63]:
importance_df.query('importance >0').index.tolist()

['tc_mseki',
 'tt_mseki',
 'levelplan',
 'fukuin',
 'magutchi',
 'setsudo_hi',
 'setsudo_kj',
 'jigata',
 'hiatari',
 'garage',
 'kobetsu1',
 'jukyo',
 'chiseki_js_hb',
 'yoto1',
 'gesui',
 'gas',
 'usui',
 'tochiuri_su',
 'joken_su',
 'hy2f_date_su',
 'hy3f_date_su',
 'road1_hk',
 'road1_sb',
 'road2_hk',
 'road2_sb',
 'road3_sb',
 'road3_fi',
 'road4_sb',
 'kaoku_um',
 'kaoku_hb',
 'yheki_kotei',
 'yheki_umu',
 'yheki_yohi',
 'gk_sho_kyori',
 'gk_chu_kyori',
 'kborjs',
 'hw_status',
 'toshikuiki1',
 'kodochiku',
 'chikukeikaku',
 'keikakuroad',
 'kaihatsukyoka',
 't53kyoka',
 'hokakyoka',
 'bokachiiki',
 'minmenseki',
 'hokakisei1',
 'kinshijiko',
 'rosenka_hb',
 'koji_hb',
 'kijun_hb',
 'mseki_rd_hb',
 'mseki_dp_hb',
 'tt_mseki_min_hb',
 'tt_mseki_max_hb',
 'tc_mseki_avg_hb',
 'fi4m_yohi',
 'bus_yohi',
 'bus_hon',
 'sho_conv',
 'sho_super',
 'sho_shoten',
 'sho_market',
 'shu_jutaku',
 'shu_park',
 'shu_shop',
 'shu_factory',
 'shu_bochi',
 'shu_kokyo',
 'shu_kaido',
 'shu_line_ari'

In [64]:
importance_df.query('importance >0').shape

(162, 1)

In [65]:
importance_df.shape

(202, 1)

In [66]:
Submit_df

,id,MAPE_sum,MAPE_0,MAPE_1,MAPE_2,MAPE_3,MAPE_4,MAPE_mean
0,test_0000,9.801211e+07,1.882709e+07,2.053789e+07,1.936538e+07,1.931356e+07,1.996818e+07,1.960242e+07
1,test_0001,1.321413e+08,2.665431e+07,2.635524e+07,2.672689e+07,2.571745e+07,2.668737e+07,2.642825e+07
2,test_0002,1.254867e+08,2.519976e+07,2.514737e+07,2.552844e+07,2.473826e+07,2.487287e+07,2.509734e+07
3,test_0003,1.381256e+08,2.773684e+07,2.815998e+07,2.744669e+07,2.769507e+07,2.708706e+07,2.762513e+07
4,test_0004,1.326665e+08,2.651164e+07,2.667575e+07,2.672998e+07,2.643297e+07,2.631614e+07,2.653330e+07
5,test_0005,1.236494e+08,2.486503e+07,2.467316e+07,2.464332e+07,2.437603e+07,2.509183e+07,2.472987e+07
6,test_0006,1.323382e+08,2.664905e+07,2.653315e+07,2.641947e+07,2.658384e+07,2.615273e+07,2.646765e+07
7,test_0007,1.256053e+08,2.550824e+07,2.514386e+07,2.503928e+07,2.486981e+07,2.504410e+07,2.512106e+07
8,test_0008,1.317335e+08,2.648538e+07,2.639080e+07,2.671028e+07,2.553514e+07,2.661193e+07,2.634671e+07
9,test_0009,1.463569e+08,2.940064e+07,2.944411e+07,2.962938e+07,2.919277e+07,2.869004e+07,2.927139e+07


In [67]:
Submit_df[['id','MAPE_mean']].to_csv('output/submit_0803_4.tsv',header=False,index=False,sep ='\t')

In [68]:
importance = importance_df.reset_index().rename(columns={'feature_name':'項目名'})

In [69]:
pd.merge(importance,data_definition).sort_values('importance',ascending=False)

,項目名,importance,データ種別,項目名（日本語）,説明,データ型
60,rosenka_hb,7.67261e+17,現場,路線価格(平米),路線価格㎡単価を収録,数値
1,tt_mseki,3.16716e+17,号棟,建物面積,建物面積㎡数,数値
123,eki_nm1,1.95104e+17,現場,駅1,主に利用する駅の名前,テキスト
62,kijun_hb,1.8379e+17,現場,基準地価(平米),基準地価㎡単価を収録,数値
5,magutchi,1.78157e+17,号棟,間口〔m〕,前面道路の間口〔m〕,数値
61,koji_hb,1.72052e+17,現場,公示地価(平米),公示地価㎡単価を収録,数値
65,tt_mseki_min_hb,1.01132e+17,現場,建物最小面積(平米),建物最小面積㎡数,数値
67,tc_mseki_avg_hb,9.03218e+16,現場,土地平均面積(平米),土地平均面積㎡数,数値
0,tc_mseki,7.48336e+16,号棟,土地面積,土地面積㎡数,数値
66,tt_mseki_max_hb,5.65857e+16,現場,建物最大面積(平米),建物最大面積㎡数,数値
